# Empowering People to Negotiate a Better Auto Loan Rate
### Nicklas Ankarstad



### Introduction  to the Problem

A major concern for lenders is being left out of the conversation when a potential borrower talks to an autodealer. If lenders could be a part of the conversation the customers is having at the dealership, they could potentially help the customer by providing a lower cost service in the form of lower interest rate or lower payments. The problem is knowing when a person is actually at a car dealership.


In many cases the car-dealers are offering the person who is buying a car a loan and many insurance policies that are significantly marked-up. By enabling financial instutitions to understand when someone is at a car dealership, they are able to help reduce the borrowers total cost of ownership a lot and gives them a better negotiating position with the dealer.


### The Data


As a prototype, Foursquare location data will be used for the Chicago area. Using this information with some coordinate data, we will calculate the distance from the dealership and create a flag whether or not we believe this individual is at a car dealership.

We will also use data from a local csv file with examples of people, their location coordinates (longitude and latitude) and their email addresses.


### Methodology

We will use the foursquare data to gather auto dealership data. We will get a flat files (csv) with people data. In a commercial setting this could potentially come from the company's web development teams. If the company has a mobile app, they may already be capturing this information. We will use the flat file as a prototype. 

Using the foursquare auto dealership data and the geopy library we will calculate the distance between each person's location and the dealership list. Anyone who is 100 feet away from the coordinates of the dealership will be included on our list being at the dealership

In [1]:
## Import packages we will use
import requests ## for getting the data
import numpy as np
import pandas as pd

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from geopy.distance import geodesic

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library


Solving environment: ...working... done

# All requested packages already installed.



In [41]:
## Set-up s we can pull the data from Foursquare
CLIENT_ID = '--' 
CLIENT_SECRET = '--'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: --
CLIENT_SECRET:--


In our example we will use the woodfield mall in Schaumburg, IL. To use this location we must first get the longitude and latitude of the mall.

In [3]:
address = "woodfield mall"

geolocator = Nominatim(user_agent="Autoloan_GeoFence")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)
#print(location)

42.04646885 -88.0369186181039


Next we will define what we are looking for. We start with searching for the keyword 'Auto Dealer' and narrow it down further by using Foursquares category ID for auto dealerships.

We also limit the results to 50 records, which is the maximum Foursquare's free accounts will return in a query.


In [4]:
search_query = 'Auto Dealer'
radius = 10000
category_id = '4eb1c1623b7b52c0e1adc2ec' ## Category ID for Auto Dealership
LIMIT = 50

We then string these things together into one long URL that we will use in our get request to pull the data.

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}&categoryId={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT, category_id)
url

'https://api.foursquare.com/v2/venues/search?client_id=UZPQFKNWHW2LN1XS455X5LQAQPWZ3RHXCYYLQUE3TCOHYEG5&client_secret=DQBNDPCWRQ3IATDRNWID5ZMKQ2GMPJ5B1F4MNTQ3G0WOXZTF&ll=42.04646885,-88.0369186181039&v=20180605&query=Auto Dealer&radius=10000&limit=50&categoryId=4eb1c1623b7b52c0e1adc2ec'

Using a get request, we pull the data from the URL and print the results below.

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c905a384c1f6746bce073f0'},
 'response': {'venues': [{'id': '5757b2aa498ec3b823f08dd1',
    'name': 'Patrick Dealer Group',
    'location': {'address': '526 Mall Dr',
     'lat': 42.0391555,
     'lng': -88.0397469,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.0391555,
       'lng': -88.0397469}],
     'distance': 847,
     'postalCode': '60173',
     'cc': 'US',
     'city': 'Schaumburg',
     'state': 'IL',
     'country': 'United States',
     'formattedAddress': ['526 Mall Dr',
      'Schaumburg, IL 60173',
      'United States']},
    'categories': [{'id': '4eb1c1623b7b52c0e1adc2ec',
      'name': 'Auto Dealership',
      'pluralName': 'Auto Dealerships',
      'shortName': 'Auto Dealer',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/automotive_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1552964152',
    'hasPerk': False},
   {'id': '4b5e1bc9f964a520117e29e3',
    'name': 

The get request returned a json file. We want to pull out the relevant parts and transform them into a dataframe so we can use it to compare against the people data.

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,5757b2aa498ec3b823f08dd1,526 Mall Dr,US,Schaumburg,United States,NaN,847,"[526 Mall Dr, Schaumburg, IL 60173, United Sta...","[{'label': 'display', 'lat': 42.0391555, 'lng'...",42.039155,-88.039747,60173,IL,Patrick Dealer Group,v-1552964152,NaN
1,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,4b5e1bc9f964a520117e29e3,750 E Golf Rd,US,Schaumburg,United States,at N Plum Grove Rd.,2025,"[750 E Golf Rd (at N Plum Grove Rd.), Schaumbu...","[{'label': 'display', 'lat': 42.05100948239872...",42.051009,-88.060645,60173,IL,Schaumburg Honda Automobiles,v-1552964152,NaN
2,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,4e3c434f1495bf24a5cac900,1230 E Golf Rd,US,Schaumburg,United States,NaN,893,"[1230 E Golf Rd, Schaumburg, IL 60173, United ...","[{'label': 'display', 'lat': 42.05027031333262...",42.050270,-88.046445,60173,IL,Chevy Dealership,v-1552964152,NaN
3,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,55524d57498e0e84c302ae60,529 A West Wise Rd,US,Schaumburg,United States,NaN,6587,"[529 A West Wise Rd, Schaumburg, IL 60193, Uni...","[{'label': 'display', 'lat': 42.00475, 'lng': ...",42.004750,-88.093409,60193,IL,All Fit Auto Body Parts,v-1552964152,NaN
4,"[{'id': '4eb1c1623b7b52c0e1adc2ec', 'name': 'A...",False,4da311ced686b60ce10ec528,333 W Rand Rd,US,Mount Prospect,United States,at Elmhurst Rd,8924,"[333 W Rand Rd (at Elmhurst Rd), Mount Prospec...","[{'label': 'display', 'lat': 42.08443, 'lng': ...",42.084430,-87.941800,60056,IL,The Autobarn Volkswagen of Mt. Prospect,v-1552964152,51342073


That dataframe is not as clean as we would want it. Let's clean it up a bit.

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

# drop the venues without category = Auto Dealership
dataframe_filtered = dataframe_filtered.loc[dataframe_filtered['categories'] == 'Auto Dealership'].reset_index(drop =True)

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Patrick Dealer Group,Auto Dealership,526 Mall Dr,US,Schaumburg,United States,NaN,847,"[526 Mall Dr, Schaumburg, IL 60173, United Sta...","[{'label': 'display', 'lat': 42.0391555, 'lng'...",42.039155,-88.039747,60173,IL,5757b2aa498ec3b823f08dd1
1,Schaumburg Honda Automobiles,Auto Dealership,750 E Golf Rd,US,Schaumburg,United States,at N Plum Grove Rd.,2025,"[750 E Golf Rd (at N Plum Grove Rd.), Schaumbu...","[{'label': 'display', 'lat': 42.05100948239872...",42.051009,-88.060645,60173,IL,4b5e1bc9f964a520117e29e3
2,Chevy Dealership,Auto Dealership,1230 E Golf Rd,US,Schaumburg,United States,NaN,893,"[1230 E Golf Rd, Schaumburg, IL 60173, United ...","[{'label': 'display', 'lat': 42.05027031333262...",42.050270,-88.046445,60173,IL,4e3c434f1495bf24a5cac900
3,All Fit Auto Body Parts,Auto Dealership,529 A West Wise Rd,US,Schaumburg,United States,NaN,6587,"[529 A West Wise Rd, Schaumburg, IL 60193, Uni...","[{'label': 'display', 'lat': 42.00475, 'lng': ...",42.004750,-88.093409,60193,IL,55524d57498e0e84c302ae60
4,The Autobarn Volkswagen of Mt. Prospect,Auto Dealership,333 W Rand Rd,US,Mount Prospect,United States,at Elmhurst Rd,8924,"[333 W Rand Rd (at Elmhurst Rd), Mount Prospec...","[{'label': 'display', 'lat': 42.08443, 'lng': ...",42.084430,-87.941800,60056,IL,4da311ced686b60ce10ec528
5,AUTO CHOICE INC,Auto Dealership,2285 S Mount Prospect Rd,US,Des Plaines,United States,NaN,10373,"[2285 S Mount Prospect Rd, Des Plaines, IL 600...","[{'label': 'display', 'lat': 42.0120486, 'lng'...",42.012049,-87.920340,60018,IL,5adaf6222347244ce248c26f
6,GT Auto Mall,Auto Dealership,2445 E Oakton St,US,Arlington Heights,United States,NaN,7893,"[2445 E Oakton St, Arlington Heights, IL 60005...","[{'label': 'display', 'lat': 42.02178757, 'lng...",42.021788,-87.947421,60005,IL,5a8a89e6419a9e0678056f11
7,Subaru dealership,Auto Dealership,Golf Rd.,US,Schaumburg,United States,NaN,6929,"[Golf Rd., Schaumburg, IL, United States]","[{'label': 'display', 'lat': 42.05374888602425...",42.053749,-88.120170,NaN,IL,4ea196a361af76b39c2c0edb
8,Old Towne Auto,Auto Dealership,128 N Bloomingdale Rd,US,Bloomingdale,United States,NaN,10283,"[128 N Bloomingdale Rd, Bloomingdale, IL 60108...","[{'label': 'display', 'lat': 41.96026, 'lng': ...",41.960260,-88.081590,60108,IL,5008643ce4b00ec56da7b516
9,Ace Auto Group,Auto Dealership,NaN,US,Des Plaines,United States,NaN,11171,"[Des Plaines, IL, United States]","[{'label': 'display', 'lat': 42.05587646625741...",42.055876,-87.902364,NaN,IL,527eaf5211d273e82ffe1113


Now that we have our list of autodealerships, lets show them on the map. Visualizing the data will help us understand how close or far away they are.

In [30]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the woodfield mall

# add a red circle marker to represent the center point coordinates
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='green',
    popup='Woodfield Mall',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.6
).add_to(venues_map)

# add the as blue circle markers are the Auto Dealerships
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Looks pretty good. Next, lets pull in the people data from the flat file

In [34]:
people_df = pd.read_csv('Longitute and Latitude Users with Emails v2.csv')
people_df

,Individual_ID,Latitude,Longitude,Email,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,555,42.05027,-88.046445,Joe_Smith@email.com,NaN,NaN,NaN,-0.04978,-0.125
1,556,42.10005,-87.921445,Random556@email.com,NaN,NaN,NaN,NaN,NaN
2,557,42.11010,-87.921945,Random557@email.com,NaN,NaN,NaN,NaN,NaN
3,558,42.12015,-87.922445,Random558@email.com,NaN,NaN,NaN,NaN,NaN
4,559,42.13020,-87.922945,Random559@email.com,NaN,NaN,NaN,NaN,NaN
5,560,42.14025,-87.923445,Random560@email.com,NaN,NaN,NaN,NaN,NaN
6,561,42.15030,-87.923945,Random561@email.com,NaN,NaN,NaN,NaN,NaN
7,562,42.16035,-87.924445,Random562@email.com,NaN,NaN,NaN,NaN,NaN
8,563,42.17040,-87.924945,Random563@email.com,NaN,NaN,NaN,NaN,NaN
9,564,42.18045,-87.927445,Random564@email.com,NaN,NaN,NaN,NaN,NaN


This dataset is pretty structured so we do not have to do much to it. Instead we will focus our effort on calculating the distance between each person's location and any of the auto dealership locations

In [35]:
min_distance_list = [] ## Create an empty list
# We want to iterate over each person's location and look up that location against each dealership location.
# We are only interested in the dealership they are closes to so we will only keep the minimum distance
for Latitude, Longitude in zip(people_df.Latitude, people_df.Longitude): 
    minimum_dist = 99999 
    for lat, lng in zip(dataframe_filtered.lat, dataframe_filtered.lng): # 
        dist = geodesic((Latitude,Longitude), (lat,lng)).miles
        if dist < minimum_dist:
            minimum_dist = dist
    min_distance_list.append(minimum_dist) # Creates a list we can append to our people dataframe
people_df['Distance from Dealership'] = min_distance_list # Creates a new column for the distance
people_df.head()

,Individual_ID,Latitude,Longitude,Email,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Distance from Dealership
0,555,42.05027,-88.046445,Joe_Smith@email.com,NaN,NaN,NaN,-0.04978,-0.125,0.000023
1,556,42.10005,-87.921445,Random556@email.com,NaN,NaN,NaN,NaN,NaN,1.502385
2,557,42.11010,-87.921945,Random557@email.com,NaN,NaN,NaN,NaN,NaN,2.044651
3,558,42.12015,-87.922445,Random558@email.com,NaN,NaN,NaN,NaN,NaN,2.658506
4,559,42.13020,-87.922945,Random559@email.com,NaN,NaN,NaN,NaN,NaN,3.304290


There is roughly 0.000568182 miles per one yard. We will set a threshold of 100 yards away from the location to alert whether or not someone is at a car dealership. Let's create a flag whether people are 100 yards away from a dealership.

In [36]:
yards_away = 100
yard_conversation = 0.000568182
dealer_flag = []
for each in people_df['Distance from Dealership']:
    #print(each)
    if each < (yards_away * yard_conversation):
        dealer_flag.append(1)
    else:
        dealer_flag.append(0)
people_df['At_Dealership'] = dealer_flag # create a new column for the binary outcome
people_df.head()



,Individual_ID,Latitude,Longitude,Email,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Distance from Dealership,At_Dealership
0,555,42.05027,-88.046445,Joe_Smith@email.com,NaN,NaN,NaN,-0.04978,-0.125,0.000023,1
1,556,42.10005,-87.921445,Random556@email.com,NaN,NaN,NaN,NaN,NaN,1.502385,0
2,557,42.11010,-87.921945,Random557@email.com,NaN,NaN,NaN,NaN,NaN,2.044651,0
3,558,42.12015,-87.922445,Random558@email.com,NaN,NaN,NaN,NaN,NaN,2.658506,0
4,559,42.13020,-87.922945,Random559@email.com,NaN,NaN,NaN,NaN,NaN,3.304290,0


### Results

The results is two individuals who are at a car dealership. This is illustrated by the map below. The red circles is the location of the people while the blue dots are the location of the auto dealerships.

We can see that two red circles overlap the blue dots and as such tell us that these people are at a dealership.




In [37]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the woodfield mall



# add the as blue circle markers are the Auto Dealerships
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)


# add the as blue circle markers are the Auto Dealerships
for Latitude, Longitude in zip(people_df.Latitude, people_df.Longitude):
    folium.features.CircleMarker(
        [Latitude,Longitude],
        radius=5,
        color='red',
#        popup=label,
        fill = True,
        fill_color='none',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

We will extract their email addresses so we can pass it over to marketing to reach out to these individuals with an offer.

In [39]:
## Return emails of people who are at the dealership
people_df.loc[people_df['At_Dealership'] == 1]['Email']


0         Joe_Smith@email.com
19    David_Johnson@email.com
Name: Email, dtype: object

### Discussion

This has been a prototype to illustrate how using geo-location and foursquare API can help determine whether someone is at a car dealership or not. A couple of items to consider if this were to be productionalized:

* Foursquare only allows 50 results to returned - may need to build internal database from pulls or switch developer account
* Used For loop which can be inefficient when scaling on large datasets - may need to use vectorization to improve performance
* The people file was a clean csv file - may not be the case in commercial application

These issues would need to be addressed before promoting to production.



### Conclusion

By using the Foursquare API with some filters, we are able to identify the auto dealerships in an area. With a second list of people and their location, we can use python and the geopy library to calculate the distance from the dealership and subsquently target them for an auto loan or insurance product.

This methodology can be applied to any marketing situation where being timely and when the location of an individual can tell what product they are looking to buy.

For the consumer, this allows them to have options in their hand when they are shopping. Often we do not know all the hidden costs and our comparison is more on the primary product we want to purchase (car in this case) and less on ancillary products that comes with that purchase (loan and insurance products). 